<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/meridian/blob/main/demo/Meridian_Getting_Started.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/meridian/blob/main/demo/Meridian_Getting_Started.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

### Before: Installation and Loading Packages

In [ ]:
# Install meridian: from PyPI @ latest release
#!pip install --upgrade google-meridian[colab,and-cuda]

# Install meridian: from PyPI @ specific version
#!pip install google-meridian[colab,and-cuda]==1.0.3

# Install meridian: from GitHub @HEAD
# !pip install --upgrade "google-meridian[colab,and-cuda] @ git+https://github.com/google/meridian.git"

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import arviz as az

import IPython

from meridian import constants
from meridian.data import load
from meridian.data import test_utils
from meridian.model import model
from meridian.model import spec
from meridian.model import prior_distribution
from meridian.analysis import optimizer
from meridian.analysis import analyzer
from meridian.analysis import visualizer
from meridian.analysis import summarizer
from meridian.analysis import formatter

# check if GPU is available
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))

# **Marketing Mix Model - Case Study with Google's Open Source Meridian**

This is a marketing mix model case study with Google's open source Meridian. The case study shows an example of a full marketing mix model analysis. All data used in this example is either open source or simulated.

The analysis is structured as follow:


1.   Project Preparation and Goal Setting
2.   Data Collection and Preparation
3.   Exploratory Data Analysis (EDA) Data visualization
4.   Modeling Model selection
5.   Results and Interpretation
6.   Recommendations and Decisions Actionable recommendations
7.   Presentation and Communication
8.   Monitoring and Model Update Monitoring
9.   Technical documentation

Even though not all steps are essential for the coding part. I would like to add some comments and recommendations. Each step is crucial to implement a successful marketing mix model, since different stakeholders should be involved.


## Project Preparation and Goal Setting.


*Purpose*

The Marketing Mix Model (MMM) analysis historical data of marketing activities and their impact on a KPI, such as sales, orders, web visits, or even brand metrics. In contrast to, for example, attribution, the KPI is not split between marketing channels to calculate the share of impact of each channel. Instead, other relevant factors, such as external factors, are also taken into account. These might include weather, holidays, economic data, or information about competitors' activities.

The purpose of MMM is to gain insights into the past performance of marketing channels while considering all these additional factors. The model can then be used for budget optimization or ROI analysis. With knowledge of the "true" performance of each marketing channel, shifting the budget from one channel to another might be optimal.

*Stakeholder*
1. Internal Data: Who are the key contact within the organization to gather all needed data?
2. External Data: Who are key contacts for external data? Is it free to use or do you need additional contracts?
3. Questions and Results: Who might be interested in results and might they have additional questions to answer within the project scope?

*Scope and timeline*

Define scope and timeline of the project. Which marketing channel will be considered? Which level of detail (channel, campaign)?
What timeline do you expect? Set specific dates each step of the project. Be kind to yourself, everthing will take longer as expected in the first place.

*Data Sources*

What data is necessaray? Think about all the data you need. Do you have access to databases for marketing data (spend, impressions, grp etc.), sales data. Which external factors might have an impact? Modeling is a iterative process. Even though you collect a lot data, you probably won't need it. Is there anything else that might be relevant for my organization?



## Data Collection and Preparation

a. Data Collection
For our case study we consider a organization in FMCG. We have online and offline marketing activies, offline focus is on TV, radio and OOH. There are a lot online activities: SEM, metasearch, display as well as social media.
The KPI of interest is sales. All **marketing activities** as well as **sales data** are covered in the Google example data.

There are five relevant **competitors**. Their impact will be tested in the model. For the case study with you sample data.

External factors are drawn from different sources. For historical weather data, we use an API from "Deutschen Wetterdienst". National bank holidays as well as school holidays are prepared with an python package.

All data is needed on a weekly level.



2. Data Collection and Preparation
Identify data sources: Gather relevant data (e.g., historical sales data, advertising spend, external factors like weather, seasonal trends, market conditions).
Data cleaning: Clean and transform the data (e.g., handling missing values, error correction, normalization).
Data integration: Combine the different data sources (e.g., sales, marketing spend, competitor data).
Data aggregation: Aggregate the data at an appropriate level (e.g., weekly, monthly) to enable better analysis.
3. Exploratory Data Analysis (EDA)
Data visualization: Initial visualizations of the data to identify trends, seasonality, and potential relationships.
Statistical analysis: Analyze correlations between marketing spend and sales figures.
Identify influencing factors: Investigate internal and external factors that affect marketing campaign success (e.g., seasonal effects, holidays).
4. Modeling
Model selection: Choose the model (e.g., linear regression, logistic regression, time series models, Bayesian models).
Model training: Train the model using historical data.
Feature engineering: Create additional features that could improve the model (e.g., interactions between marketing channels, lag effects of marketing actions).
Hyperparameter tuning: Optimize the model (e.g., selecting model parameters).
Validation and testing: Validate the model with a test dataset to assess the model’s quality (e.g., using cross-validation).
5. Results and Interpretation
Analyze model outputs: Interpret the results of the model (e.g., the impact of each marketing channel on sales, ROI of each campaign).
Seasonality and lag effects analysis: Investigate time-related effects and delayed responses to marketing efforts.
Benchmarking: Compare the results with other companies or industry benchmarks to validate the findings.
Sensitivity analysis: Assess the stability of the model under different assumptions and parameter values.
6. Recommendations and Decisions
Actionable recommendations: Derive optimization opportunities based on the model results (e.g., reallocating budgets, adjusting marketing strategies).
Budget optimization: Provide recommendations for optimal allocation of the marketing budget.
Long-term planning: Recommendations for future marketing strategies and actions to improve performance in the long run.
7. Presentation and Communication
Create report: Summarize results and recommendations in a structured report.
Presentation to stakeholders: Present the results to management or other stakeholders (e.g., with visual representations of findings and clear action points).
Communicate uncertainties: Highlight the uncertainties and limitations of the model (e.g., assumptions made in the model).
8. Monitoring and Model Update
Monitoring: Continuously monitor marketing actions and their impact on sales in real-time.
Model updates: Regularly update the model to account for new data and changing market conditions.
A/B testing: Run experiments to test the effects of specific changes and improve the model.
9. Documentation
Technical documentation: Provide detailed descriptions of methods, algorithms, and steps used in modeling.
Code documentation: Comment the code so that the model can be understood and followed by other team members.
Data sources and assumptions: Document the data sources and underlying assumptions used.


### Weather

In [ ]:
'''import os
import pickle
import pandas as pd
import glob
from DWD_hist_weather import tageswerte_land

# Pfad für die Pickle-Dateien
pickle_directory = '/home/tui/s3synced/csv_data/pickle/'


# Lösche alle Pickle-Dateien im Ordner
pickle_directory = "./csv_data/pickle/"
for file in glob.glob(os.path.join(pickle_directory, "*.pickle")):
    os.remove(file)
    print(f"Gelöscht: {file}")


# Liste der Bundesländer
bundeslaender = ['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg',
                 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen', 'Nordrhein-Westfalen',
                 'Rheinland-Pfalz', 'Saarland', 'Sachsen', 'Sachsen-Anhalt', 'Schleswig-Holstein',
                 'Thüringen']

# Leeres DataFrame für den Gesamtdatensatz
gesamtdatensatz = pd.DataFrame()

# Erstelle den Ordner für Pickle-Dateien, falls er nicht existiert
os.makedirs(pickle_directory, exist_ok=True)

# Iteriere durch alle Bundesländer
for bundesland in bundeslaender:
    print(f"\nVerarbeite Wetterdaten für: {bundesland}...")
    pickle_dateiname = os.path.join(pickle_directory, f"{bundesland}.pickle")

    try:
        # Lade Wetterdaten aus Pickle-Datei
        with open(pickle_dateiname, 'rb') as file:
            tageswerte = pickle.load(file)
        print(f"Wetterdaten für {bundesland} aus Pickle-Datei geladen.")
    except (FileNotFoundError, EOFError):
        # Lade Wetterdaten neu, falls Pickle-Datei nicht existiert
        print(f"Pickle-Datei nicht gefunden. Lade Wetterdaten für {bundesland} neu...")
        try:
            tageswerte = tageswerte_land(bundesland, still=True)
            # Speichere die Daten als Pickle-Datei
            with open(pickle_dateiname, 'wb') as file:
                pickle.dump(tageswerte, file)
            print(f"Wetterdaten für {bundesland} gespeichert in {pickle_dateiname}.")
        except Exception as e:
            print(f"Fehler beim Abrufen der Wetterdaten für {bundesland}: {e}")
            continue

    # Füge die Wetterdaten zum Gesamtdatensatz hinzu
    tageswerte['Bundesland'] = bundesland
    gesamtdatensatz = pd.concat([gesamtdatensatz, tageswerte])

# Zeige die ersten Zeilen des kombinierten Datensatzes an
print("\nKombinierte Wetterdaten:")
print(gesamtdatensatz.tail())
'''



In [ ]:
'''import pandas as pd

def aggregiere_woechentlich(gesamtdatensatz):
    # Stelle sicher, dass 'Datum' als Spalte vorhanden ist
    if gesamtdatensatz.index.name == 'Datum':
        gesamtdatensatz = gesamtdatensatz.reset_index()

    # Konvertiere Datum zur Wochenspalte (Montag der Woche) und nenne sie 'Datum'
    gesamtdatensatz['Datum'] = gesamtdatensatz['Datum'].dt.to_period('W').apply(lambda r: r.start_time)

    # Gruppierung auf Wochenebene
    wochenaggregat = gesamtdatensatz.groupby('Datum').agg({
        'TempMean': 'mean',
        'TempMax': 'mean',
        'TempMin': 'mean',
        'HumidityMean': 'mean',
        'SunshineDuration': 'mean',
        }).reset_index()

    # Entferne die Hilfsspalten
    df_seas = wochenaggregat[['Datum', 'seas_TempMean', 'seas_TempMax', 'seas_TempMin', 'seas_HumidityMean', 'seas_SunshineDuration']]

    return df_seas

df_seas = aggregiere_woechentlich(gesamtdatensatz)

    # Ausgabe des DataFrames
print("\nWöchentlicher Datensatz:")
print(df_seas.head())

    # Optional: Speichere den Datensatz
#df_seas.to_csv("woechentliche_wetterdaten.csv", index=False)

'''

In [ ]:
'''import ferien
import pandas as pd
import time
from requests.exceptions import HTTPError

def create_pivot_table():
    retries = 5
    backoff_factor = 2

    # Retry mechanism with exponential backoff
    for i in range(retries):
        try:
            vacations_list = ferien.all_vacations()
            break
        except HTTPError as e:
            if e.response.status_code == 429:  # Too Many Requests
                sleep_time = backoff_factor ** i
                print(f"Rate limit exceeded. Retrying in {sleep_time} seconds...")
                time.sleep(sleep_time)
            else:
                raise
        except Exception as e:
            raise RuntimeError("An error occurred: " + str(e))
    else:
        raise RuntimeError("API request failed after several retries.")

    # Tabelle mit Einwohnerzahlen der Bundesländer
    population_dict = {
        'BW': 11124642, 'BY': 13176989, 'BE': 3677472, 'BB': 2537868,
        'HB': 676463, 'HH': 1853935, 'HE': 6295017, 'MV': 1611160,
        'NI': 8027031, 'NW': 17924591, 'RP': 4106485, 'SL': 982348,
        'SN': 4043002, 'ST': 2169253, 'SH': 2922005, 'TH': 2108863
    }

    # Liste von Listen für DataFrame erstellen
    processed_data = []
    for vacation in vacations_list:
        start_date = vacation.start
        end_date = vacation.end
        name = vacation.name
        state = vacation.state_code

        current_date = start_date
        while current_date <= end_date:
            week_start_date = current_date - pd.Timedelta(days=current_date.weekday())
            week_end_date = week_start_date + pd.Timedelta(days=6)
            vacation_days = min((week_end_date - current_date).days + 1, 7)

            population = population_dict.get(state, 1)
            processed_data.append([week_start_date.date(), name, state, vacation_days, vacation_days * population])
            current_date += pd.Timedelta(weeks=1)

    df = pd.DataFrame(processed_data, columns=['Datum', 'name', 'state', 'Ferientage', 'hlds_Schoolholidaysgew'])
    df['Datum'] = pd.to_datetime(df['Datum'])
    df_ferien = df.pivot_table(index='Datum', columns='state', values='Ferientage', aggfunc='sum', fill_value=0)
    total_population = sum(population_dict.values())
    df_ferien = df_ferien.apply(lambda x: x / population_dict[x.name] * total_population, axis=0)
    df_ferien['hlds_Schoolholidaysgew'] = df_ferien.sum(axis=1)

    return df_ferien

if __name__ == '__main__':
    df_ferien = create_pivot_table()
    print(df_ferien.tail())

df_main = pd.merge(df_main, df_ferien['hlds_Schoolholidaysgew'], on='Datum', how='left')'''

'''df_ferien = pd.read_csv('/home/tui/s3synced/csv_data/df_main.csv')

df_ferien = df_ferien[['Datum','hlds_Schoolholidaysgew']]
df_ferien['Datum'] = pd.to_datetime(df_ferien['Datum'], format='%Y-%m-%d')

df_main = pd.merge(df_main, df_ferien, on='Datum', how='left')'''


In [ ]:
## Bundesweite Feiertage

import holidays
import pandas as pd

# Erstellen einer Liste von deutschen Feiertagen für die Jahre 2021 bis 2024
de_holidays = holidays.Germany(years=[2021, 2022, 2023, 2024])

# Erstellen eines Datumsbereichs vom 1. Mai 2021 bis zum 30. April 2024
date_range = pd.date_range(start='2021-05-01', end='2024-12-31', freq='D')

# Initialisieren eines leeren DataFrame
df = pd.DataFrame(index=date_range)
df.index = df.index.date

# Überprüfen, ob ein Datum ein Feiertag ist, und Zuweisen von 1 für Feiertag und 0 für Nicht-Feiertag
df["hlds_bankholidays"] = [1 if date in de_holidays else 0 for date in df.index]

# Hinzufügen von Isokalenderwoche und Jahr
df["iso_year"] = pd.to_datetime(df.index).isocalendar().year
df["iso_week"] = pd.to_datetime(df.index).isocalendar().week

# Gruppieren nach Isokalenderwoche und Jahr und Berechnen der Summe der Feiertage pro Woche
df_feiertage = df.groupby(["iso_year", "iso_week"])["hlds_bankholidays"].sum().reset_index()

# Berechnen des Montags der jeweiligen Woche
df_feiertage["Datum"] = pd.to_datetime(df_feiertage["iso_year"].astype(str) + '-W' + df_feiertage["iso_week"].astype(str) + '-1', format='%G-W%V-%u')

# Spalten umbenennen um die Originalnamen beizubehalten
df_feiertage = df_feiertage.rename(columns={"iso_year": "Jahr", "iso_week": "Kalenderwoche"})

# Ergebnis anzeigen
df_feiertage


'''
# Erstellen der Variable 'easter' und 'christmas'
df_feiertage['hlds_easter'] = [1 if any(de_holidays.get(date) in ['Good Friday', 'Easter Monday'] for date in pd.date_range(start=week, end=week + pd.DateOffset(days=6))) else 0 for week in df_feiertage['Datum']]
# Erstellen der Variable 'easter'
# Erstellen der Variable 'christmas'
df_feiertage['hlds_christmas'] = [1 if any("christmas" in de_holidays.get(date, "") for date in pd.date_range(start=week, end=week + pd.DateOffset(days=6))) else 0 for week in df_feiertage['Datum']]
'''
df_feiertage.head()
# Merge df_main mit df_feiertage anhand der 'Datum'-Spalte
df_feiertage['Datum'] = pd.to_datetime(df_feiertage['Datum'], format='%Y-%m-%d')
df_main = pd.merge(df_main, df_feiertage, left_on='Datum', right_on='Datum', how='left')


# **Introduction to Meridian Demo**

Welcome to the Meridian end-to-end demo. This simplified demo showcases the fundamental functionalities and basic usage of the library, including working examples of the major modeling steps:


<ol start="0">
  <li><a href="#install">Install</a></li>
  <li><a href="#load-data">Load the data</a></li>
  <li><a href="#configure-model">Configure the model</a></li>
  <li><a href="#model-diagnostics">Run model diagnostics</a></li>
  <li><a href="#generate-summary">Generate model results & two-page output</a></li>
  <li><a href="#generate-optimize">Run budget optimization & two-page output</a></li>
  <li><a href="#save-model">Save the model object</a></li>
</ol>


Note that this notebook skips all of the exploratory data analysis and preprocessing steps. It assumes that you have completed these tasks before reaching this point in the demo.

This notebook utilizes sample data. As a result, the numbers and results obtained might not accurately reflect what you encounter when working with a real dataset.

<a name="install"></a>
## Step 0: Install

1\. Make sure you are using one of the available GPU Colab runtimes which is **required** to run Meridian. You can change your notebook's runtime in `Runtime > Change runtime type` in the menu. All users can use the T4 GPU runtime which is sufficient to run the demo colab, free of charge. Users who have purchased one of Colab's paid plans have access to premium GPUs (such as V100, A100 or L4 Nvidia GPU).

2\. Install the latest version of Meridian, and verify that GPU is available.

<a name="load-data"></a>
## Step 1: Load the data

Load the [simulated dataset in CSV format](https://github.com/google/meridian/blob/main/meridian/data/simulated_data/csv/geo_all_channels.csv) as follows.

1\. Map the column names to their corresponding variable types. For example, the column names 'GQV' and 'Competitor_Sales' are mapped to `controls`. The required variable types are `time`, `controls`, `population`, `kpi`, `revenue_per_kpi`, `media` and `spend`. If your data includes organic media or non-media treatments, you can add them using `organic_media` and `non_media_treatments` arguments. For the definition of each variable, see
[Collect and organize your data](https://developers.google.com/meridian/docs/user-guide/collect-data).

In [ ]:
coord_to_columns = load.CoordToColumns(
    time='time',
    geo='geo',
    controls=['GQV', 'Competitor_Sales'],
    population='population',
    kpi='conversions',
    revenue_per_kpi='revenue_per_conversion',
    media=[
        'Channel0_impression',
        'Channel1_impression',
        'Channel2_impression',
        'Channel3_impression',
        'Channel4_impression',
    ],
    media_spend=[
        'Channel0_spend',
        'Channel1_spend',
        'Channel2_spend',
        'Channel3_spend',
        'Channel4_spend',
    ],
    organic_media=['Organic_channel0_impression'],
    non_media_treatments=['Promo'],
)

2\. Map the media variables and the media spends to the designated channel names intended for display in the two-page HTML output. In the following example,  'Channel0_impression' and 'Channel0_spend' are connected to the same channel, 'Channel0'.

In [ ]:
correct_media_to_channel = {
    'Channel0_impression': 'Channel_0',
    'Channel1_impression': 'Channel_1',
    'Channel2_impression': 'Channel_2',
    'Channel3_impression': 'Channel_3',
    'Channel4_impression': 'Channel_4',
}
correct_media_spend_to_channel = {
    'Channel0_spend': 'Channel_0',
    'Channel1_spend': 'Channel_1',
    'Channel2_spend': 'Channel_2',
    'Channel3_spend': 'Channel_3',
    'Channel4_spend': 'Channel_4',
}

3\. Load the CSV data using `CsvDataLoader`. Note that `csv_path` is the path to the data file location.

In [ ]:
loader = load.CsvDataLoader(
    csv_path="https://raw.githubusercontent.com/google/meridian/refs/heads/main/meridian/data/simulated_data/csv/geo_all_channels.csv",
    kpi_type='non_revenue',
    coord_to_columns=coord_to_columns,
    media_to_channel=correct_media_to_channel,
    media_spend_to_channel=correct_media_spend_to_channel,
)
data = loader.load()

Note that the simulated data here does not contain reach and frequency. We recommend including reach and frequency data whenever they are available. For information about the advantages of utilizing reach and frequency, see [Bayesian Hierarchical Media Mix Model Incorporating Reach and Frequency Data](https://research.google/pubs/bayesian-hierarchical-media-mix-model-incorporating-reach-and-frequency-data/#:~:text=By%20incorporating%20R%26F%20into%20MMM,based%20on%20optimal%20frequency%20recommendations.). For code snippet for loading reach and frequency data, see [Load geo-level data with reach and frequency](https://developers.google.com/meridian/docs/user-guide/load-geo-data-with-rf)

The documentation provides guidance for instances where reach and frequency data is accessible for specific channels. Additionally, for information about how to load other data types and formats, including data with reach and frequency, see [Supported data types and formats](https://developers.google.com/meridian/docs/user-guide/supported-data-types-formats).

<a name="configure-model"></a>
## Step 2: Configure the model

Meridian uses Bayesian framework and Markov Chain Monte Carlo (MCMC) algorithms to sample from the posterior distribution.

1\. Inititalize the `Meridian` class by passing the loaded data and the customized model specification. One advantage of Meridian lies in its capacity to calibrate the model directly through ROI priors, as described in [Media Mix Model Calibration With Bayesian Priors](https://research.google/pubs/media-mix-model-calibration-with-bayesian-priors/). In this particular example, the ROI priors for all media channels are identical, with each being represented as Lognormal(0.2, 0.9).

In [ ]:
roi_mu = 0.2     # Mu for ROI prior for each media channel.
roi_sigma = 0.9  # Sigma for ROI prior for each media channel.
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior)

mmm = model.Meridian(input_data=data, model_spec=model_spec)

2\. Use the `sample_prior()` and `sample_posterior()` methods to obtain samples from the prior and posterior distributions of model parameters. If you are using the T4 GPU runtime this step may take about 10 minutes for the provided data set.

In [ ]:
%%time
mmm.sample_prior(500)
mmm.sample_posterior(n_chains=7, n_adapt=500, n_burnin=500, n_keep=1000)

For more information about configuring the parameters and using a customized model specification, such as setting different ROI priors for each media channel, see [Configure the model](https://developers.google.com/meridian/docs/user-guide/configure-model).

<a name="model-diagnostics"></a>
## Step 3: Run model diagnostics

After the model is built, you must assess convergence, debug the model if needed, and then assess the model fit.

1\. Assess convergence. Run the following code to generate r-hat statistics. R-hat close to 1.0 indicate convergence. R-hat < 1.2 indicates approximate convergence and is a reasonable threshold for many problems.

In [ ]:
model_diagnostics = visualizer.ModelDiagnostics(mmm)
model_diagnostics.plot_rhat_boxplot()

2\. Assess the model's fit by comparing the expected sales against the actual sales.

In [ ]:
model_fit = visualizer.ModelFit(mmm)
model_fit.plot_model_fit()

For more information and additional model diagnostics checks, see [Modeling diagnostics](https://developers.google.com/meridian/docs/user-guide/model-diagnostics).

<a name="generate-summary"></a>
## Step 4: Generate model results & two-page output

To export the two-page HTML summary output, initialize the `Summarizer` class with the model object. Then pass in the filename, filepath, start date, and end date to `output_model_results_summary` to run the summary for that time duration and save it to the specified file.

In [ ]:
mmm_summarizer = summarizer.Summarizer(mmm)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
filepath = '/content/drive/MyDrive'
start_date = '2021-01-25'
end_date = '2024-01-15'
mmm_summarizer.output_model_results_summary('summary_output.html', filepath, start_date, end_date)

Here is a preview of the two-page output based on the simulated data:

In [ ]:
IPython.display.HTML(filename='/content/drive/MyDrive/summary_output.html')

For a customized two-page report, model results summary table, and individual visualizations, see [Model results report](https://developers.google.com/meridian/docs/user-guide/generate-model-results-report) and [plot media visualizations](https://developers.google.com/meridian/docs/user-guide/plot-media-visualizations).





<a name="generate-optimize"></a>
## Step 5: Run budget optimization & generate an optimization report

You can choose what scenario to run for the budget allocation. In default scenario, you find the optimal allocation across channels for a given budget to maximize the return on investment (ROI).

1\. Instantiate the `BudgetOptimizer` class and run the `optimize()` method without any customization, to run the default library's Fixed Budget Scenario to maximize ROI.

In [ ]:
%%time
budget_optimizer = optimizer.BudgetOptimizer(mmm)
optimization_results = budget_optimizer.optimize()

2\. Export the 2-page HTML optimization report, which contains optimized spend allocations and ROI.

In [ ]:
filepath = '/content/drive/MyDrive'
optimization_results.output_optimization_summary('optimization_output.html', filepath)

In [ ]:
IPython.display.HTML(filename='/content/drive/MyDrive/optimization_output.html')

For information about customized optimization scenarios, such as flexible budget scenarios, see [Budget optimization scenarios](https://developers.google.com/meridian/docs/user-guide/budget-optimization-scenarios). For more information about optimization results summary and individual visualizations, see [optimization results output](https://developers.google.com/meridian/docs/user-guide/generate-optimization-results-output) and [optimization visualizations](https://developers.google.com/meridian/docs/user-guide/plot-optimization-visualizations).

<a name="save-model"></a>
## Step 6: Save the model object

We recommend that you save the model object for future use. This helps you to  avoid repetitive model runs and saves time and computational resources. After the model object is saved, you can load it at a later stage to continue the analysis or visualizations without having to re-run the model.


Run the following codes to save the model object:

In [ ]:
file_path='/content/drive/MyDrive/saved_mmm.pkl'
model.save_mmm(mmm, file_path)

Run the following codes to load the saved model:

In [ ]:
mmm = model.load_mmm(file_path)